<a href="https://colab.research.google.com/github/tobischw/audio-super-res/blob/experiments/Audio_Super_Resolution_(Python2)_Speaker1_only.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Audio Super-Resolution Notebook

## Connect Google Drive

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

%cd 'gdrive/My Drive/audio-super-res'

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
/content/gdrive/My Drive/audio-super-res


# Download Dataset

In [ ]:
%cd /content/gdrive/My Drive/audio-super-res/data/vctk
!make

Streaming output truncated to the last 5000 lines.
VCTK-Corpus/wav48/p340/p340_189.wav
VCTK-Corpus/wav48/p340/p340_190.wav
VCTK-Corpus/wav48/p340/p340_191.wav
VCTK-Corpus/wav48/p340/p340_192.wav
VCTK-Corpus/wav48/p340/p340_193.wav
VCTK-Corpus/wav48/p340/p340_194.wav
VCTK-Corpus/wav48/p340/p340_195.wav
VCTK-Corpus/wav48/p340/p340_196.wav
VCTK-Corpus/wav48/p340/p340_197.wav
VCTK-Corpus/wav48/p340/p340_198.wav
VCTK-Corpus/wav48/p340/p340_199.wav
VCTK-Corpus/wav48/p340/p340_200.wav
VCTK-Corpus/wav48/p340/p340_201.wav
VCTK-Corpus/wav48/p340/p340_202.wav
VCTK-Corpus/wav48/p340/p340_203.wav
VCTK-Corpus/wav48/p340/p340_204.wav
VCTK-Corpus/wav48/p340/p340_205.wav
VCTK-Corpus/wav48/p340/p340_206.wav
VCTK-Corpus/wav48/p340/p340_207.wav
VCTK-Corpus/wav48/p340/p340_208.wav
VCTK-Corpus/wav48/p340/p340_209.wav
VCTK-Corpus/wav48/p340/p340_210.wav
VCTK-Corpus/wav48/p340/p340_211.wav
VCTK-Corpus/wav48/p340/p340_212.wav
VCTK-Corpus/wav48/p340/p340_213.wav
VCTK-Corpus/wav48/p340/p340_214.wav
VCTK-Corpus/w

## Install Dependencies

In [2]:
!pip install tensorflow==1.13.1 keras==1.2.1 numpy==1.16.4 scipy==1.2.1 librosa==0.4.3 joblib==0.11 h5py==2.9.0 matplotlib==1.5.1

     |████████████████████████████████| 92.5MB 50kB/s 
     |████████████████████████████████| 174kB 48.2MB/s 
     |████████████████████████████████| 24.8MB 126kB/s 
     |████████████████████████████████| 1.5MB 43.0MB/s 
     |████████████████████████████████| 184kB 51.9MB/s 
     |████████████████████████████████| 2.8MB 41.5MB/s 
     |████████████████████████████████| 54.0MB 49kB/s 
     |████████████████████████████████| 368kB 46.8MB/s 
     |████████████████████████████████| 3.2MB 44.2MB/s 
  Created wheel for keras: filename=Keras-1.2.1-cp27-none-any.whl size=207855 sha256=65da2e2e03cbd2ab6fbcf4fb0c37e740a7ecb20953530837e87057aacf9577d4
  Stored in directory: /root/.cache/pip/wheels/72/89/fe/faa5b6078ee30f61c1ff9d49b4d755e533084feaea6e8e8315
  Created wheel for librosa: filename=librosa-0.4.3-cp27-none-any.whl size=1537961 sha256=d2b158b331450d5f0bfa2d0772666d030011b391d90bd7ea746af10b91812ac3
  Stored in directory: /root/.cache/pip/wheels/0b/48/9d/50a6edd63d5b2865f3d6df9390acb3

In [3]:
!pip install 'joblib==0.11' --force-reinstall

     |████████████████████████████████| 184kB 2.8MB/s 
  Found existing installation: joblib 0.12.5
    Uninstalling joblib-0.12.5:
      Successfully uninstalled joblib-0.12.5


## Create Dataset
This runs pre-processing steps to create a low-res version of the high-res audio file.

**This step should be replaced with my customized downsampler that allows lets you add noise etc... (WIP)**

In [15]:
%cd /content/gdrive/My Drive/audio-super-res/data/vctk/speaker1

sca = 4
sr = 16000

tr_dim = 8192
tr_str = 4096

va_dim = 8192
va_str = 4096

train_output_name = 'vctk-speaker1-train.%d.%d.%d.%d.h5' % (sca, sr, tr_dim, tr_str)
val_output_name = 'vctk-speaker1-val.%d.%d.%d.%d.h5' % (sca, sr, va_dim, va_str)

!python ../prep_vctk.py --file-list speaker1-train-files.txt --in-dir ../VCTK-Corpus/wav48/p225 --out $train_output_name --scale $sca --dimension $tr_dim --stride $tr_str --interpolate --low-pass
!python ../prep_vctk.py --file-list speaker1-val-files.txt --in-dir ../VCTK-Corpus/wav48/p225 --out $val_output_name --scale $sca --dimension $va_dim --stride $va_str --interpolate --low-pass

/content/gdrive/My Drive/audio-super-res/data/vctk/speaker1
223
0/223
10/223
20/223
30/223
40/223
50/223
60/223
70/223
80/223
90/223
100/223
110/223
120/223
130/223
140/223
150/223
160/223
170/223
180/223
190/223
200/223
210/223
220/223
3328
32
0/32
10/32
20/32
30/32
384


## Train the Model

In [16]:
%cd /content/gdrive/My Drive/audio-super-res/src

!python run.py train --train ../data/vctk/speaker1/$train_output_name --val ../data/vctk/speaker1/$val_output_name -e 120 --batch-size 64 --lr 3e-4 --logname singlespeaker --model audiotfilm --r 4 --layers 4 --piano false --pool_size 8 --strides 8 --full true

/content/gdrive/My Drive/audio-super-res/src
Using TensorFlow backend.
List of arrays in input file: [u'data', u'label']
Shape of X: (3328, 8192, 1)
Shape of Y: (3328, 8192, 1)
List of arrays in input file: [u'data', u'label']
Shape of X: (384, 8192, 1)
Shape of Y: (384, 8192, 1)
audiotfilm
building model...
Instructions for updating:
Colocations handled automatically by placer.
Traceback (most recent call last):
  File "run.py", line 172, in <module>
    main()
  File "run.py", line 169, in main
    args.func(args)
  File "run.py", line 119, in train
    model = get_model(args, n_dim, r, from_ckpt=False, train=True)
  File "run.py", line 155, in get_model
    strides=args.strides, opt_params=opt_params, log_prefix=args.logname)  
  File "/content/gdrive/My Drive/audio-super-res/src/models/audiotfilm.py", line 29, in __init__
    opt_params=opt_params, log_prefix=log_prefix)
  File "/content/gdrive/My Drive/audio-super-res/src/models/model.py", line 48, in __init__
    self.predictions